<a href="https://colab.research.google.com/github/drummond8scott/tokyo_dataset/blob/main/Tokyo_dataset_de.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import required packages for loading, transforming and exporting tokyo data
import openpyxl
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import (col, upper, trim, regexp_replace, when, count,
                                 initcap, concat_ws, split, to_timestamp, coalesce, isnan, when, length, trim,
                                 sum as spark_sum, countDistinct, desc, expr)
from pyspark.sql.window import Window
from pyspark.sql.types import *

In [5]:
# Load Tokyo dataset excel files from github into dataframes
base_url = "https://raw.githubusercontent.com/drummond8scott/tokyo_dataset/main/"

# Load datasets
athletes_df = pd.read_excel(base_url + "Athletes.xlsx")
coaches_df = pd.read_excel(base_url + "Coaches.xlsx")
medals_df = pd.read_excel(base_url + "Medals.xlsx")
teams_df = pd.read_excel(base_url + "Teams.xlsx")

# Print counts for loaded datasets
print("Dataset Sizes:")
print(f"Athletes: {len(athletes_df)} records")
print(f"Coaches: {len(coaches_df)} records")
print(f"Medals: {len(medals_df)} records")
print(f"Teams: {len(teams_df)} records")

# Print first 3 rows of each dataset
print("\nFirst few rows of each dataset:")
for name, df in [("Athletes", athletes_df), ("Coaches", coaches_df),
                 ("Medals", medals_df), ("Teams", teams_df)]:
    print(f"\n{name} dataset:")
    print(df.head(3))

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Dataset Sizes:
Athletes: 11085 records
Coaches: 394 records
Medals: 93 records
Teams: 743 records

First few rows of each dataset:

Athletes dataset:
                Name     NOC           Discipline
0    AALERUD Katrine  Norway         Cycling Road
1        ABAD Nestor   Spain  Artistic Gymnastics
2  ABAGNALE Giovanni   Italy               Rowing

Coaches dataset:
              Name    NOC  Discipline Event
0  ABDELMAGID Wael  Egypt    Football   NaN
1        ABE Junya  Japan  Volleyball   NaN
2    ABE Katsuhiko  Japan  Basketball   NaN

Medals dataset:
   Rank                    Team/NOC  Gold  Silver  Bronze  Total  \
0     1    United States of America    39      41      33    113   
1     2  People's Republic of China    38      32      18     88   
2     3                       Japan    27      14      17     58   

   Rank by Total  
0              1  
1              2  
2              5  

Teams dataset:
      Name      Discipline                         NOC  Event
0  Belgium  

In [7]:
# Convert pandas dataframes to PySpark dataframes

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Tokyo Olympics Analysis") \
    .getOrCreate()

athletes_df = spark.createDataFrame(athletes_df)
coaches_df = spark.createDataFrame(coaches_df)
medals_df = spark.createDataFrame(medals_df)
teams_df = spark.createDataFrame(teams_df)

In [8]:


# Athletes DataFrame Analysis
print("=== ATHLETES DATASET ANALYSIS ===")
total_athletes = athletes_df.count()
print(f"Total athletes: {total_athletes}")

# Check missing values in athletes
for column in athletes_df.columns:
    null_count = athletes_df.filter(col(column).isNull()).count()
    if null_count > 0:
        print(f"Missing values in {column}: {null_count} ({(null_count/total_athletes*100):.2f}%)")

# Check for duplicate athletes
duplicate_athletes = athletes_df.groupBy(athletes_df.columns).count().filter("count > 1")
if duplicate_athletes.count() > 0:
    print(f"\nFound {duplicate_athletes.count()} duplicate athlete entries")


# Check NOC distribution
noc_counts = athletes_df.groupBy("NOC").count().orderBy("count", ascending=False)
print("\nTop 5 NOCs by athlete count:")
noc_counts.show(5)

print("\n=== MEDALS DATASET ANALYSIS ===")
total_medal_records = medals_df.count()
print(f"Total medal records: {total_medal_records}")

# Check missing values in medals
for column in medals_df.columns:
    null_count = medals_df.filter(col(column).isNull()).count()
    if null_count > 0:
        print(f"Missing values in {column}: {null_count} ({(null_count/total_medal_records*100):.2f}%)")

# Check medal counts by type - Modified to handle separate medal columns
medal_distribution = medals_df.select(
    sum(col("Gold")).alias("Gold"),
    sum(col("Silver")).alias("Silver"),
    sum(col("Bronze")).alias("Bronze")
).withColumn("Total", col("Gold") + col("Silver") + col("Bronze"))

print("\nMedal distribution:")
medal_distribution.show()

# Cross-reference check between athletes and medals
athlete_countries = athletes_df.select("NOC").distinct().count()
medal_countries = medals_df.select("Team/NOC").distinct().count()
print(f"\nUnique countries in athletes: {athlete_countries}")
print(f"Unique countries in medals: {medal_countries}")

print("\n=== DATA CONSISTENCY CHECKS ===")
# Check for any NOCs in medals that don't appear in athletes
medals_nocs = set(medals_df.select("Team/NOC").distinct().collect())
athletes_nocs = set(athletes_df.select("NOC").distinct().collect())
mismatched_nocs = medals_nocs - athletes_nocs
if mismatched_nocs:
    print(f"NOCs in medals but not in athletes: {mismatched_nocs}")

# Check for unusual values in key fields
print("\nUnique disciplines:")
athletes_df.select("Discipline").distinct().show(truncate=False)

print("\nSample of unusual names (if any):")
athletes_df.filter(~col("Name").rlike("^[A-Za-z\\s\\-\\']+$")).show(5)

=== ATHLETES DATASET ANALYSIS ===
Total athletes: 11085

Found 1 duplicate athlete entries

Top 5 NOCs by athlete count:
+--------------------+-----+
|                 NOC|count|
+--------------------+-----+
|United States of ...|  615|
|               Japan|  586|
|           Australia|  470|
|People's Republic...|  401|
|             Germany|  400|
+--------------------+-----+
only showing top 5 rows


=== MEDALS DATASET ANALYSIS ===
Total medal records: 93

Medal distribution:
+----+------+------+-----+
|Gold|Silver|Bronze|Total|
+----+------+------+-----+
| 340|   338|   402| 1080|
+----+------+------+-----+


Unique countries in athletes: 206
Unique countries in medals: 93

=== DATA CONSISTENCY CHECKS ===

Unique disciplines:
+---------------------+
|Discipline           |
+---------------------+
|Tennis               |
|Boxing               |
|Marathon Swimming    |
|Golf                 |
|Rowing               |
|Baseball/Softball    |
|Judo                 |
|Sailing           

In [9]:
from pyspark.sql.functions import *

# Get unique NOC and Team combinations from each dataset
print("=== NOC/Team Analysis ===\n")

# Athletes Dataset Analysis
athlete_teams = athletes_df.select("NOC").distinct()
print(f"Number of unique NOC-Team combinations in Athletes: {athlete_teams.count()}")

# Medals Dataset Analysis
medal_teams = medals_df.select("Team/NOC").distinct()
print(f"Number of unique NOC-Team combinations in Medals: {medal_teams.count()}")

# Coaches Dataset Analysis
coach_teams = coaches_df.select("NOC").distinct()
print(f"Number of unique NOC-Team combinations in Coaches: {coach_teams.count()}")

# Show discrepancies in Athletes dataset
print("\n=== Athletes Dataset: Cases where NOC doesn't match expected Team pattern ===")
athletes_df.select("NOC") \
    .distinct() \
    .orderBy("NOC") \
    .show(truncate=False)

# Compare NOCs across datasets
athlete_nocs = set([row.NOC for row in athletes_df.select("NOC").distinct().collect()])
medal_nocs = set([row["Team/NOC"] for row in medals_df.select("Team/NOC").distinct().collect()])
coach_nocs = set([row.NOC for row in coaches_df.select("NOC").distinct().collect()])

print("\n=== NOC Comparison Across Datasets ===")
print(f"Total unique NOCs in Athletes: {len(athlete_nocs)}")
print(f"Total unique NOCs in Medals: {len(medal_nocs)}")
print(f"Total unique NOCs in Coaches: {len(coach_nocs)}")

# Find NOCs that appear in one dataset but not others
print("\n=== NOC Discrepancies ===")
print("NOCs in Medals but not in Athletes:", medal_nocs - athlete_nocs if medal_nocs - athlete_nocs else "None")
print("NOCs in Athletes but not in Medals:", athlete_nocs - medal_nocs if athlete_nocs - medal_nocs else "None")
print("NOCs in Coaches but not in Athletes:", coach_nocs - athlete_nocs if coach_nocs - athlete_nocs else "None")
print("NOCs in Athletes but not in Coaches:", athlete_nocs - coach_nocs if athlete_nocs - coach_nocs else "None")

# Analyze Team naming patterns
print("\n=== Team Name Pattern Analysis ===")
print("Sample of Team names from Athletes dataset:")
athletes_df.select("NOC") \
    .distinct() \
    .orderBy(rand()) \
    .show(10, truncate=False)

print("\nSample of Team names from Medals dataset:")
medals_df.select("Team/NOC") \
    .distinct() \
    .orderBy(rand()) \
    .show(10, truncate=False)

print("\nSample of Team names from Coaches dataset:")
coaches_df.select("NOC") \
    .distinct() \
    .orderBy(rand()) \
    .show(10, truncate=False)

# Check for any unusual characters or patterns in NOC codes
print("\n=== Unusual NOC Patterns ===")
print("Athletes Dataset - NOCs not following standard 3-letter pattern:")
athletes_df.filter(~col("NOC").rlike("^[A-Z]{3}$")) \
    .select("NOC") \
    .distinct() \
    .show()

print("\nMedals Dataset - NOCs not following standard 3-letter pattern:")
medals_df.filter(~col("Team/NOC").rlike("^[A-Z]{3}$")) \
    .select("Team/NOC") \
    .distinct() \
    .show()

print("\nCoaches Dataset - NOCs not following standard 3-letter pattern:")
coaches_df.filter(~col("NOC").rlike("^[A-Z]{3}$")) \
    .select("NOC") \
    .distinct() \
    .show()

=== NOC/Team Analysis ===

Number of unique NOC-Team combinations in Athletes: 206
Number of unique NOC-Team combinations in Medals: 93
Number of unique NOC-Team combinations in Coaches: 61

=== Athletes Dataset: Cases where NOC doesn't match expected Team pattern ===
+-------------------+
|NOC                |
+-------------------+
|Afghanistan        |
|Albania            |
|Algeria            |
|American Samoa     |
|Andorra            |
|Angola             |
|Antigua and Barbuda|
|Argentina          |
|Armenia            |
|Aruba              |
|Australia          |
|Austria            |
|Azerbaijan         |
|Bahamas            |
|Bahrain            |
|Bangladesh         |
|Barbados           |
|Belarus            |
|Belgium            |
|Belize             |
+-------------------+
only showing top 20 rows


=== NOC Comparison Across Datasets ===
Total unique NOCs in Athletes: 206
Total unique NOCs in Medals: 93
Total unique NOCs in Coaches: 61

=== NOC Discrepancies ===
NOCs in Me

In [6]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import (col, count, when, isnan, countDistinct,
                                 length, regexp_replace, trim, desc)
from pyspark.sql.types import *
from functools import reduce
def analyze_tokyo_data_quality(athletes_df, coaches_df, medals_df, teams_df):
    """
    Comprehensive data quality analysis using PySpark
    """

    def analyze_null_values(df, table_name):
        """Analyze null and empty values in each column"""
        print(f"\n=== Null Value Analysis for {table_name} ===")

        # Get total row count
        total_rows = df.count()

        # Calculate null counts and percentages for each column
        for column in df.columns:
            null_count = df.filter(
                col(column).isNull() |
                (col(column) == '') |
                isnan(column)
            ).count()

            null_percentage = (null_count / total_rows) * 100

            print(f"\nColumn: {column}")
            print(f"Null/Empty Count: {null_count}")
            print(f"Null/Empty Percentage: {null_percentage:.2f}%")

    def analyze_duplicates(df, table_name):
        """Analyze duplicate records"""
        print(f"\n=== Duplicate Analysis for {table_name} ===")

        # Count total and distinct rows
        total_rows = df.count()
        distinct_rows = df.distinct().count()
        duplicate_rows = total_rows - distinct_rows
        duplication_rate = duplicate_rows / total_rows * 100 if total_rows else 0.0

        print(f"Total Rows: {total_rows}")
        print(f"Unique Rows: {distinct_rows}")
        print(f"Duplicate Rows: {duplicate_rows}")
        print(f"Duplication Rate: {duplication_rate}%")

        # Show example duplicates
        if duplicate_rows > 0:
            print("\nSample Duplicate Records:")
            df.groupBy(df.columns) \
              .count() \
              .filter(col("count") > 1) \
              .orderBy(desc("count")) \
              .show(20, truncate=False)

    def analyze_data_patterns(df, table_name):
        """Analyze patterns in string columns"""
        print(f"\n=== Data Pattern Analysis for {table_name} ===")

        for column in df.columns:
            # Only analyze string columns
            if isinstance(df.schema[column].dataType, StringType):
                print(f"\nColumn: {column}")

                # Check for whitespace issues
                whitespace_count = df.filter(
                    (trim(col(column)) != col(column)) &
                    col(column).isNotNull()
                ).count()

                print(f"Records with leading/trailing whitespace: {whitespace_count}")

                # Length distribution
                print("\nLength Distribution:")
                df.groupBy(length(col(column)).alias("length")) \
                  .count() \
                  .orderBy("length") \
                  .show(5)

    def analyze_value_distributions(df, table_name):
        """Analyze value distributions in columns"""
        print(f"\n=== Value Distribution Analysis for {table_name} ===")

        for column in df.columns:
            print(f"\nTop values in {column}:")
            df.groupBy(column) \
              .count() \
              .orderBy(desc("count")) \
              .show(20, truncate=False)

    def analyze_referential_integrity():
        """Check referential integrity between tables"""
        print("\n=== Referential Integrity Analysis ===")

        # Athletes - Medals relationship
        orphaned_medals = medals_df \
            .join(athletes_df,
                  medals_df["Team/NOC"] == athletes_df.NOC,
                  "left_anti") \
            .count()

        print(f"\nMedals without matching Athletes: {orphaned_medals}")

        # Team consistency
        invalid_teams_athletes = athletes_df \
            .join(teams_df,
                  athletes_df.NOC == teams_df.NOC,
                  "left_anti") \
            .count()

        invalid_teams_coaches = coaches_df \
            .join(teams_df,
                  coaches_df.NOC == teams_df.NOC,
                  "left_anti") \
            .count()

        print(f"Athletes with invalid Teams: {invalid_teams_athletes}")
        print(f"Coaches with invalid Teams: {invalid_teams_coaches}")

    def analyze_data_completeness():
        """Analyze completeness of records"""
        print("\n=== Data Completeness Analysis ===")

        # Athletes completeness
        total_athletes = athletes_df.count()
        athletes_with_medals = medals_df.select("Team/NOC").distinct().count()
        athletes_with_coaches = athletes_df \
            .join(coaches_df,
                  ["NOC", "Discipline"],
                  "inner") \
            .select(athletes_df.Name).distinct().count()

        print("\nAthletes Data Completeness:")
        print(f"Total Athletes: {total_athletes}")
        print(f"Athletes with Medals: {athletes_with_medals}")
        print(f"Athletes with Coaches: {athletes_with_coaches}")

        # Calculate coverage percentages
        medal_coverage = (athletes_with_medals / total_athletes) * 100
        coach_coverage = (athletes_with_coaches / total_athletes) * 100

        print(f"Medal Coverage: {medal_coverage}%")
        print(f"Coach Coverage: {coach_coverage}%")

    def generate_summary_metrics():
        """Generate summary metrics for all tables"""
        print("\n=== Summary Quality Metrics ===")

        tables = {
            "Athletes": athletes_df,
            "Coaches": coaches_df,
            "Medals": medals_df,
            "Teams": teams_df
        }

        for table_name, df in tables.items():
            print(f"\n{table_name} Table Metrics:")
            total_rows = df.count()
            null_rows = df.filter(
                reduce(lambda x, y: x | y,
                      [col(c).isNull() for c in df.columns])
            ).count()
            duplicate_rows = total_rows - df.distinct().count()

            print(f"Total Records: {total_rows}")
            print(f"Complete Records: {total_rows - null_rows}")
            print(f"Records with Nulls: {null_rows} ({null_rows/total_rows * 100}%)")
            print(f"Duplicate Records: {duplicate_rows} ({duplicate_rows/total_rows * 100}%)")

    # Execute all analyses
    print("Starting comprehensive data quality analysis...\n")

    # Run individual analyses
    for df, name in [(athletes_df, "Athletes"),
                     (coaches_df, "Coaches"),
                     (medals_df, "Medals"),
                     (teams_df, "Teams")]:
        analyze_null_values(df, name)
        analyze_duplicates(df, name)
        analyze_data_patterns(df, name)
        analyze_value_distributions(df, name)

    # Run cross-table analyses
    analyze_referential_integrity()
    analyze_data_completeness()

    # Generate summary
    generate_summary_metrics()

# Example usage:
if __name__ == "__main__":
    # Initialize Spark Session if not already initialized
    spark = SparkSession.builder \
        .appName("Tokyo Olympics Data Quality Analysis") \
        .getOrCreate()

    # Assuming DataFrames are already loaded
    analyze_tokyo_data_quality(athletes_df, coaches_df, medals_df, teams_df)

Starting comprehensive data quality analysis...


=== Null Value Analysis for Athletes ===

Column: Name
Null/Empty Count: 0
Null/Empty Percentage: 0.00%

Column: NOC
Null/Empty Count: 0
Null/Empty Percentage: 0.00%

Column: Discipline
Null/Empty Count: 0
Null/Empty Percentage: 0.00%

=== Duplicate Analysis for Athletes ===
Total Rows: 11085
Unique Rows: 11084
Duplicate Rows: 1
Duplication Rate: 0.009021199819576003%

Sample Duplicate Records:
+-----------+-------+----------+-----+
|Name       |NOC    |Discipline|count|
+-----------+-------+----------+-----+
|ALI Mohamed|Bahrain|Handball  |2    |
+-----------+-------+----------+-----+


=== Data Pattern Analysis for Athletes ===

Column: Name
Records with leading/trailing whitespace: 1

Length Distribution:
+------+-----+
|length|count|
+------+-----+
|     4|    4|
|     5|   13|
|     6|   24|
|     7|   58|
|     8|  117|
+------+-----+
only showing top 5 rows


Column: NOC
Records with leading/trailing whitespace: 0

Length Distrib